In [1]:
from cesm_le2_recipe import recipe

/glade/u/home/mgrover/miniconda3/envs/cesm2-marbl/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


* Add pr, add directory cesm2-le in staged-recipes/recipes
## Setup to run the recipe

Set the source and target paths

In [3]:
from pathlib import Path

# Where the files are stored locally
direct = '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/T'

# Where to write the zarr stores to
target_dir = '/glade/scratch/mgrover/cesm2-le'

# Where to cache metadata
meta_dir = '/glade/scratch/mgrover/cesm2-le-metadata'

In [4]:
from fsspec.implementations.local import LocalFileSystem
from pangeo_forge_recipes.storage import FSSpecTarget, MetadataTarget

fs_local = LocalFileSystem()

cache_dir = tempfile.TemporaryDirectory()
cache_target = FSSpecTarget(fs_local, direct)

target = FSSpecTarget(fs_local, target_dir)

meta_store = MetadataTarget(fs_local, meta_dir)

recipe.input_cache = cache_target
recipe.target = target
recipe.metadata_cache = meta_store

cache_target.root_path, target.root_path, meta_store.root_path

('/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/T',
 '/glade/scratch/mgrover/cesm2-le',
 '/glade/scratch/mgrover/tmp4kmvzcxz')

### Cache metadata

In [ ]:
for input_name in recipe.iter_inputs():
    recipe.cache_input_metadata(input_name)

### Prepare the target
Basically starts the zarr store - writes the first time step

In [5]:
recipe.prepare_target()

FileNotFoundError: [Errno 2] No such file or directory: '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/T/8a8393edfcf123a1af2c1322cc5e77cc-glade_campaign_cgd_cesm_cesm2-le_timeseries_atm_proc_tseries_month_1_t_b.e21.bhistcmip6.f09_g17.le2-1161.009.cam.h0.t.201001-201412.nc'